## Initialization

In [ ]:
#@title
! git clone https://github.com/proroklab/VectorizedMultiAgentSimulator.git

In [ ]:
#@title
%cd /content/VectorizedMultiAgentSimulator

!pip install -r requirements.txt
!apt-get update
!apt-get install -y x11-utils 
!apt-get install -y xvfb
!apt-get install -y imagemagick
!pip install -e .

In [ ]:
#@title
!pip install pyvirtualdisplay
import pyvirtualdisplay
display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
display.start()

## Run


In [ ]:
#  Copyright (c) 2022.
#  ProrokLab (https://www.proroklab.org/)
#  All rights reserved.

import time
import random
import torch
from PIL import Image

from vmas import make_env

scenario_name = "waterfall"

# Scenario specific variables
n_agents = 4

num_envs = 32  # Number of vectorized environments
continuous_actions = False
device = "cpu"  # or cuda or any other torch device
n_steps = 100  # Number of steps before returning done
dict_spaces = True  # Weather to return obs, rewards, and infos as dictionaries with agent names (by default they are lists of len # of agents)

simple_2d_action = (
    [0, 0.5] if continuous_actions else [3]
)  # Simple action tell each agent to go down

env = make_env(
    scenario=scenario_name,
    num_envs=num_envs,
    device=device,
    continuous_actions=continuous_actions,
    dict_spaces=dict_spaces,
    wrapper=None,
    seed=None,
    # Environment specific variables
    n_agents=n_agents,
)

frame_list = []  # For creating a gif
init_time = time.time()
step = 0

for s in range(n_steps):
    step += 1
    print(f"Step {step}")

    # VMAS actions can be either a list of tensors (one per agent)
    # or a dict of tensors (one entry per agent with its name as key)
    # Both action inputs can be used independently of what type of space its chosen
    dict_actions = random.choice([True, False])

    actions = {} if dict_actions else []
    for i, agent in enumerate(env.agents):
        action = torch.tensor(
            simple_2d_action,
            device=device,
        ).repeat(num_envs, 1)
        if dict_actions:
            actions.update({agent.name: action})
        else:
            actions.append(action)

    obs, rews, dones, info = env.step(actions)

    frame_list.append(
        Image.fromarray(env.render(mode="rgb_array", agent_index_focus=None))
    )  # Can give the camera an agent index to focus on

gif_name = scenario_name + ".gif"

# Produce a gif
frame_list[0].save(
    gif_name,
    save_all=True,
    append_images=frame_list[1:],
    duration=3,
    loop=0,
)


total_time = time.time() - init_time
print(
    f"It took: {total_time}s for {n_steps} steps of {num_envs} parallel environments on device {device} "
    f"for {scenario_name} scenario."
)

In [ ]:
from IPython.display import Image
Image(open(f'{scenario_name}.gif','rb').read())

In [ ]:
import os
# Requires imagemagick to be installed to convert the gif in faster format
os.system(f"convert -delay 1x30 -loop 0 {gif_name} {scenario_name}_fast.gif")
from IPython.display import Image
Image(open(f'{scenario_name}_fast.gif','rb').read())